## Imports

In [340]:
# imports libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

In [341]:
# read dataset
df = pd.read_csv('dataset.csv')

## EDA

In [342]:
df.head(5)

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers
0,10-Sep-12,7,5,0,2,2,9
1,NaN,8,6,0,2,2,14
2,NaN,9,8,3,0,0,14
3,NaN,10,9,2,0,1,14
4,NaN,11,11,1,4,4,11


In [343]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Date              15 non-null     object
 1   Time (Local)      336 non-null    int64 
 2   Eyeballs          336 non-null    int64 
 3   Zeroes            336 non-null    int64 
 4   Completed Trips   336 non-null    int64 
 5   Requests          336 non-null    int64 
 6   Unique Drivers    336 non-null    int64 
dtypes: int64(6), object(1)
memory usage: 18.5+ KB


In [344]:
df.shape

(336, 7)

In [345]:
# show all columns
df.columns

Index(['Date', 'Time (Local)', 'Eyeballs ', 'Zeroes ', 'Completed Trips ',
       'Requests ', 'Unique Drivers'],
      dtype='object')

In [346]:
# Apply trim to all Headings
df.columns = df.columns.str.strip()

In [347]:
# fill NaN values with the first value
df['Date'] = df['Date'].ffill(axis=0)

In [348]:
# Convert object type to datetime
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y')

In [349]:
df['Date'].value_counts()

Date
2012-09-11    24
2012-09-12    24
2012-09-13    24
2012-09-14    24
2012-09-15    24
2012-09-16    24
2012-09-17    24
2012-09-18    24
2012-09-19    24
2012-09-20    24
2012-09-21    24
2012-09-22    24
2012-09-23    24
2012-09-10    17
2012-09-24     7
Name: count, dtype: int64

На 10 и 24 числа присутствуют не все данные за все сутки

## Analyze

#### # 1 - Выясним в какой день было совершено наибольшее кол-во поездок?

In [350]:
df.groupby('Date', as_index=False)['Completed Trips'].sum().nlargest(1, 'Completed Trips')

,Date,Completed Trips
12,2012-09-22,248


*Как видим, 22 сентября 2012 года было совершено наибольшее кол-во поездок!*

#### # 2 - Какое самое наибольшее количество поездок было совершено в течение 24 часов?

In [351]:
# define function to create timestamp from columns 'Date' and 'Time (Local)'
def create_timestamp(date, time):
    return pd.to_datetime(f'{date} {time}:00')


# create Series Timestamp
df['Timestamp'] = df.apply(lambda row: create_timestamp(row['Date'], row['Time (Local)']), axis=1)

In [352]:
# Create Index with fixed window frame
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=24)
df_2 = df.set_index('Timestamp').select_dtypes(include=['number']).rolling(window=indexer, min_periods=1).sum()

In [353]:
# Create Column Time Interval with string type
time_format = '%Y-%m-%d %H:%M'
df_2['Time Interval'] = df_2.index.strftime(time_format) + ' - ' + (df_2.index + pd.Timedelta('24 hours')).strftime(
    time_format)

In [354]:
time_interval, completed_trips = df_2.nlargest(1, 'Completed Trips')[['Time Interval', 'Completed Trips']].iloc[0]
print(f'В 24 часовой интервал {time_interval} было совершено наибольшее количество поездок - {completed_trips}')

В 24 часовой интервал 2012-09-21 17:00 - 2012-09-22 17:00 было совершено наибольшее количество поездок - 278.0


#### # 3 - Выясним в какое время суток было больше всего запросов?

In [355]:
df_3 = df.groupby(['Time (Local)'], as_index=False).agg(avg_req_cnt=('Requests', 'mean')).sort_values('avg_req_cnt',
                                                                                                      ascending=False)

In [356]:
# calculate percentage of avg_req_cnt by Time
df_3['pct'] = df_3['avg_req_cnt'] / df_3['avg_req_cnt'].sum() * 100

In [357]:
# calculate cumulative sum in descending order of pct values
df_3['cum_total'] = df_3['pct'].sort_values(ascending=False).cumsum()
df_3

,Time (Local),avg_req_cnt,pct,cum_total
23,23,13.142857,9.903122,9.903122
22,22,12.428571,9.364909,19.268030
19,19,11.142857,8.396125,27.664155
0,0,10.142857,7.642626,35.306781
18,18,8.500000,6.404736,41.711518
21,21,8.000000,6.027987,47.739505
20,20,7.642857,5.758881,53.498385
2,2,7.142857,5.382131,58.880517
17,17,7.000000,5.274489,64.155005
1,1,6.857143,5.166846,69.321851


*В период времени с 18:00 до 01:00 делается более 50% всех суточных заказов*

#### # 4 - Рассчитаем какой процент всех пустых поездок за двухнедельный период пришелся на выходные (с 17:00 Пятницы до 15:00 Воскресенья)

In [358]:
# total zeroes in the dataset
total_zeroes = df['Zeroes'].sum()
print('Total zeroes in dataset: ', total_zeroes)

Total zeroes in dataset:  1429


In [359]:
weekend_zeroes_pct = (df[((df['Date'].dt.dayofweek == 4) & (df['Time (Local)'] >= 17)) |
                          (df['Date'].dt.dayofweek == 5) |
                          ((df['Date'].dt.dayofweek == 6) & (df['Time (Local)'] < 3))][
                           'Zeroes'].sum() / total_zeroes * 100).round(2)
print(f'Процент пустых заказов в выходные: {weekend_zeroes_perc} %')

Процент пустых заказов в выходные: 44.86 %


#### # 5 - Каково средневзвешенное соотношение выполненных поездок на одного водителя за двухнедельный период?

In [360]:
# Calculate ratio
df['completed_trip_per_driver'] = df['Completed Trips'] / df['Unique Drivers']

In [361]:
# Drop rows with NaN
df_5 = df[df['Unique Drivers'] > 0]

In [362]:
# Calculate wighted average
w_av_ratio_without_zero_drivers = np.average(df_5['completed_trip_per_driver'], weights=df_5['Completed Trips'])
print('Weighted Average Ratio Without Zero Unique Drivers: ', w_av_ratio_without_zero_drivers)

Weighted Average Ratio Without Zero Unique Drivers:  0.8282775715092395


In [363]:
normal_av_ratio_without_zero_drivers = np.average(df_5['completed_trip_per_driver'])
print('Normal Average Ratio Without Zero Unique Drivers: ', normal_av_ratio_without_zero_drivers)

Normal Average Ratio Without Zero Unique Drivers:  0.4743397876937869


#### # 6 - В какой 8 часовой интервал времени поступило больше всего заказов? (за какую дату)

In [364]:
# set Timestamp as Index and resample as 8h Interval with sum all values
df_6 = df.set_index('Timestamp').select_dtypes(include=['number']).resample('8H').sum()

In [365]:
# Create Column Time Interval with string type
time_format = '%Y-%m-%d %H:%M:%S'
df_6['Time Interval'] = df_6.index.strftime(time_format) + ' - ' + (df_6.index + pd.Timedelta('8 hours')).strftime(
    time_format)

In [366]:
# Choose maximum value
df_6.nlargest(1, 'Requests')

,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,completed_trip_per_driver,Time Interval
Timestamp,,,,,,,,
2012-09-21 16:00:00,156,515,88,153,192,198,6.357265,2012-09-21 16:00:00 - 2012-09-22 00:00:00


*В данный интервал времени - 2012-09-21 16:00:00 - 2012-09-22 00:00:00 поступило наибольшее количество запросов.*

#### # 7 - Увеличивается ли предложение водителей при росте спроса на поездки?